#Very important
# Face Unlock Application Access Instructions

# 
# To access the Streamlit Face Unlock Application:
# 1.  **Run this entire Kaggle Notebook from top to bottom.**
#     Ensure all cells execute successfully.
#  2.  **Wait patiently during the "Phase 3: Running Streamlit App" section.**
#     You will see continuous "--- Current Streamlit Log Tail" messages.
#     This indicates the application is starting and loading models.
#     This process can take 1-3 minutes depending on the Kaggle instance.
# 3.  **Look for the `ngrok` Public URL.**
#     Once Streamlit is fully launched, the notebook will print the `ngrok` tunnel URL,
#     which will look something like:
#     `Your ngrok URL is: https://<random-characters>.ngrok-free.app`
#  4.  **Click on this `ngrok` URL.**
#     This is the ONLY link that will work to access your Streamlit app from outside Kaggle.
#     (Do NOT use `localhost` or internal IP addresses provided in Streamlit's own logs.)
#  5.  **Keep the Kaggle Notebook cell running.**
#     The cell will continuously print a message like "[HH:MM:SS] Kaggle cell active..."
#     **You MUST keep this cell running for the Streamlit app to remain online and accessible.**
#     If you stop the cell, the app will go offline immediately.
# 6.  **To stop the application and release resources:**
#     Simply click the "Stop" button (red square) on the top-right of this Kaggle cell.
#     The notebook will then perform a clean shutdown.


In [3]:
import os
import shutil
import time
from subprocess import Popen, PIPE, STDOUT
import sys
import glob
import gc # Import garbage collector
import tensorflow as tf # Import tensorflow here for memory config
import threading # Still need threading for ngrok tunnel
import select # For checking if pipe has data (though less relevant with nohup direct logging)

# --- Cleanup Step: Terminate previous processes ---
print("--- Cleanup: Shutting down old processes... ---")
# Use killall for a more aggressive cleanup of any lingering Streamlit/ngrok processes
# The `|| true` prevents errors if the process isn't found
!fuser -k -9 8501/tcp || true
!pkill -f ngrok || true
!killall -q streamlit || true # Ensure any old streamlit processes are gone
print("Cleanup complete. Starting new session.")

# --- Step 0: Setup ---
print("\n--- Setup: Installing/Upgrading dependencies. This will take some time. ---")
# Prioritize your core dependencies.
!{sys.executable} -m pip install -q --upgrade \
    streamlit==1.36.0 \
    opencv-python-headless \
    numpy \
    tensorflow \
    Pillow \
    h5py \
    scikit-learn \
    av \
    requests \
    tqdm \
    pyngrok

print("Dependency installation/upgrade attempt complete.")

# --- Configure TensorFlow Memory Growth ---
# This helps prevent OOM errors by allocating GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow: Memory growth enabled for GPUs.")
    except RuntimeError as e:
        print(f"TensorFlow: Could not set memory growth: {e}")
else:
    print("TensorFlow: No GPU devices found or configured.")

# Clear TensorFlow session and collect garbage to free up memory
tf.keras.backend.clear_session()
gc.collect()

# --- Verify pyngrok installation ---
print("Verifying pyngrok installation...")
try:
    from pyngrok import ngrok
    print("pyngrok imported successfully.")
except ImportError:
    print("Error: pyngrok could not be imported. Please ensure it installed correctly.")
    exit()

# --- Step 1: Define Paths and Prepare Files ---
print("\n--- Step 1: Preparing files for app launch ---")

# Define base paths for artifacts and app
# !!! IMPORTANT: YOU MUST CORRECT THIS PATH BASED ON YOUR KAGGLE DATASET LOCATION !!!
# Use `!ls -R /kaggle/input/` in a separate cell on Kaggle to find the exact path.
# Example: If your dataset is named 'my-face-models' and files are at its root,
# ARTIFACTS_INPUT_PATH = '/kaggle/input/my-face-models/'
ARTIFACTS_INPUT_PATH = '/kaggle/input/lma/pytorch/default/1/' # <--- REVIEW AND CORRECT THIS LINE

APP_ROOT_KAGGLE = '/kaggle/working/face_unlock_app/'
MODELS_DIR_KAGGLE = os.path.join(APP_ROOT_KAGGLE, 'models')
DNN_MODELS_DIR = os.path.join(MODELS_DIR_KAGGLE, 'dnn_face_detector_models')
DB_PATH = os.path.join(APP_ROOT_KAGGLE, 'face_db.db')

# Add a cleanup step for the database file
print("Cleaning up old database file...")
try:
    if os.path.exists(DB_PATH):
        os.remove(DB_PATH)
        print(f"Old database file '{DB_PATH}' removed successfully.")
    else:
        print("No old database file found. Starting fresh.")
except OSError as e:
    print(f"Warning: Could not remove database file. The app will attempt to fix the schema instead.")

# Create destination directories
os.makedirs(APP_ROOT_KAGGLE, exist_ok=True)
os.makedirs(MODELS_DIR_KAGGLE, exist_ok=True)
os.makedirs(DNN_MODELS_DIR, exist_ok=True)

# Define full, absolute paths for all files
LIVENESS_MODEL_SRC = os.path.join(ARTIFACTS_INPUT_PATH, 'liveness_model.h5')
LIVENESS_MODEL_DST = os.path.join(MODELS_DIR_KAGGLE, 'liveness_model.h5')

FACE_EMBEDDING_MODEL_SRC = os.path.join(ARTIFACTS_INPUT_PATH, 'face_embedding_model.h5')
FACE_EMBEDDING_MODEL_DST = os.path.join(MODELS_DIR_KAGGLE, 'face_embedding_model.h5')

FACE_DB_SRC = os.path.join(ARTIFACTS_INPUT_PATH, 'face_db.db')
FACE_DB_DST = os.path.join(APP_ROOT_KAGGLE, 'face_db.db')

CAFFE_MODEL_SRC = os.path.join(ARTIFACTS_INPUT_PATH, 'res10_300x300_ssd_iter_140000_fp16.caffemodel')
CAFFE_MODEL_DST = os.path.join(DNN_MODELS_DIR, 'res10_300x300_ssd_iter_140000_fp16.caffemodel')

PROTOTXT_SRC = os.path.join(ARTIFACTS_INPUT_PATH, 'deploy.prototxt')
PROTOTXT_DST = os.path.join(DNN_MODELS_DIR, 'deploy.prototxt')

# Copy artifacts to their destinations
try:
    print(f"Attempting to copy liveness model from: {LIVENESS_MODEL_SRC} to {LIVENESS_MODEL_DST}")
    shutil.copy(LIVENESS_MODEL_SRC, LIVENESS_MODEL_DST)
    print(f"Attempting to copy face embedding model from: {FACE_EMBEDDING_MODEL_SRC} to {FACE_EMBEDDING_MODEL_DST}")
    shutil.copy(FACE_EMBEDDING_MODEL_SRC, FACE_EMBEDDING_MODEL_DST)
    print(f"Attempting to copy face detector caffemodel from: {CAFFE_MODEL_SRC} to {CAFFE_MODEL_DST}")
    shutil.copy(CAFFE_MODEL_SRC, CAFFE_MODEL_DST)
    print(f"Attempting to copy face detector prototxt from: {PROTOTXT_SRC} to {PROTOTXT_DST}")
    shutil.copy(PROTOTXT_SRC, PROTOTXT_DST)
    
    if os.path.exists(FACE_DB_SRC):
        print(f"Attempting to copy existing database from: {FACE_DB_SRC} to {FACE_DB_DST}")
        shutil.copy(FACE_DB_SRC, FACE_DB_DST)
    else:
        print(f"Database source file not found at {FACE_DB_SRC}. A new one will be created.")
    
    print("All necessary files attempted to be copied.")

except FileNotFoundError as e:
    print(f"Error: A required file was not found during copy. Please check your source paths.")
    print(f"Details: {e}")
    # Exit if critical files cannot be found at source
    exit()
except Exception as e:
    print(f"An unexpected error occurred during file copying: {e}")
    exit()

# --- Verification Step ---
print("Verifying model files exist in destination...")
required_files = [LIVENESS_MODEL_DST, CAFFE_MODEL_DST, PROTOTXT_DST]
all_found = True
for f in required_files:
    if not os.path.exists(f):
        print(f"ERROR: Required file not found in destination: {f}")
        all_found = False
    else:
        print(f"Verified: {f} exists.")
if not all_found:
    print("Some required model files are missing. Please fix input paths or dataset attachments.")
    exit()
print("All required model files verified in destination.")


# --- Step 2: Generating Python files ---
print("\n--- Step 2: Generating Python files ---")

# We'll generate utils.py and app.py using full absolute paths
models_abs_path = os.path.abspath(MODELS_DIR_KAGGLE)
app_root_abs_path = os.path.abspath(APP_ROOT_KAGGLE)

utils_code = f"""
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_v2_preprocess
import h5py
import sqlite3
import streamlit as st
import datetime
import gc # Import garbage collector

# Clear TensorFlow session to free up memory (redundant if already cleared in main script, but safe)
tf.keras.backend.clear_session()
gc.collect()

# Define model paths using the absolute paths passed from the main script
MODELS_PATH_GLOBAL = "{models_abs_path}"

# Database file path using absolute path
DB_PATH = "{os.path.abspath(os.path.join(app_root_abs_path, 'face_db.db'))}"

# Database Manager Class
class SQLiteDatabaseManager:
    def __init__(self, db_path=DB_PATH):
        self.db_path = db_path
        self._initialize_db()

    def _initialize_db(self):
        conn = self._get_connection()
        c = conn.cursor()

        c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='users'")
        table_exists = c.fetchone()

        if table_exists:
            c.execute("PRAGMA table_info(users)")
            columns = [col[1] for col in c.fetchall()]
            if 'registration_date' not in columns:
                print("Adding missing 'registration_date' column to 'users' table.")
                c.execute("ALTER TABLE users ADD COLUMN registration_date TEXT")
        else:
            c.execute('''
                CREATE TABLE users (
                    id INTEGER PRIMARY KEY,
                    username TEXT UNIQUE NOT NULL,
                    embedding BLOB NOT NULL,
                    registration_date TEXT
                )
            ''')

        c.execute('''
            CREATE TABLE IF NOT EXISTS history (
                id INTEGER PRIMARY KEY,
                username TEXT,
                event_type TEXT,
                timestamp TEXT,
                success INTEGER
            )
        ''')
        conn.commit()
        conn.close()

    def _get_connection(self):
        return sqlite3.connect(self.db_path)

    def register_user(self, username, embedding):
        try:
            conn = self._get_connection()
            c = conn.cursor()
            embedding_blob = sqlite3.Binary(embedding)
            registration_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            c.execute("INSERT INTO users (username, embedding, registration_date) VALUES (?, ?, ?)", 
                      (username, embedding_blob, registration_date))
            conn.commit()
            conn.close()
            return True, "User registered successfully."
        except sqlite3.IntegrityError:
            return False, "Username already exists."
        except Exception as e:
            return False, f"Database error: {{e}}"

    def get_user_embedding(self, username):
        conn = self._get_connection()
        c = conn.cursor()
        c.execute("SELECT embedding FROM users WHERE username = ?", (username,))
        result = c.fetchone()
        conn.close()
        if result:
            return np.frombuffer(result[0], dtype=np.float32)
        return None

    def get_all_users(self):
        conn = self._get_connection()
        c = conn.cursor()
        c.execute("SELECT username, registration_date FROM users")
        users = c.fetchall()
        conn.close()
        return users
    
    def log_event(self, username, event_type, success):
        conn = self._get_connection()
        c = conn.cursor()
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        c.execute("INSERT INTO history (username, event_type, timestamp, success) VALUES (?, ?, ?, ?)", 
                  (username, event_type, timestamp, int(success)))
        conn.commit()
        conn.close()

    def get_history(self, username=None):
        conn = self._get_connection()
        c = conn.cursor()
        if username:
            c.execute("SELECT username, event_type, timestamp, success FROM history WHERE username = ?", (username,))
        else:
            c.execute("SELECT username, event_type, timestamp, success FROM history")
        history = c.fetchall()
        conn.close()
        return history

# Model and Detector Loading Functions using absolute paths
@st.cache_resource
def load_face_embedding_model():
    # *** FIX: Changed alpha to 0.35 as 0.25 is not available with imagenet weights ***
    # This should resolve the ValueError and is the smallest pre-trained model.
    base_model = MobileNetV2(
        input_shape=(224, 224, 3), 
        include_top=False, 
        weights='imagenet', # Retaining ImageNet weights
        alpha=0.35 # Changed from 0.25 to 0.35 (smallest available with imagenet)
    )
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    embedding_model = Model(inputs=base_model.input, outputs=x)
    return embedding_model

@st.cache_resource
def load_liveness_model():
    return None 

@st.cache_resource
def load_face_detector(prototxt_path, model_path):
    return cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

# Core ML Logic Functions
def detect_and_align_face(frame, net):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            face_width = endX - startX
            face_height = endY - startY
            aspect_ratio = face_width / face_height
            if not (0.7 < aspect_ratio < 1.3): 
                return None, None

            face = frame[startY:endY, startX:endX]
            if face.shape[0] == 0 or face.shape[1] == 0: 
                continue
            face = cv2.resize(face, (224, 224))
            return face, (startX, startY, endX, endY)
    return None, None

def get_face_embedding(face, embedding_model):
    face = face.astype('float32')
    face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face_processed = mobilenet_v2_preprocess(face_rgb)
    face_expanded = np.expand_dims(face_processed, axis=0)
    embedding = embedding_model.predict(face_expanded, verbose=0)[0]
    embedding = embedding / np.linalg.norm(embedding)
    return embedding

def check_liveness(face_image):
    gray = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    _, thresh = cv2.threshold(blurred, 220, 255, cv2.THRESH_BINARY) 
    
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 50 and area < 500: 
                (x, y), radius = cv2.minEnclosingCircle(contour)
                if cv2.arcLength(contour, True) > 0:
                    circularity = (4 * np.pi * area) / (cv2.arcLength(contour, True) ** 2)
                    if circularity > 0.6: 
                        return False, "Suspicious reflection detected. Please remove any shiny objects."
    return True, "Liveness check passed."
    
def verify_face(known_embedding, new_embedding, threshold=0.5):
    distance = np.linalg.norm(known_embedding - new_embedding)
    is_match = distance < threshold
    return is_match, distance
"""

app_code = f"""
import streamlit as st
import cv2
import numpy as np
from utils import (
    load_face_embedding_model, 
    load_face_detector, 
    check_liveness,
    verify_face,
    SQLiteDatabaseManager,
    detect_and_align_face,
    get_face_embedding
)
import os

# Define model paths using the absolute paths passed from the main script
MODELS_DIR = "{models_abs_path}"
DNN_MODELS_DIR = "{os.path.join(models_abs_path, 'dnn_face_detector_models')}"

# Load models and database manager only once using Streamlit's caching
face_embedding_model = load_face_embedding_model()
face_detector = load_face_detector(os.path.join(DNN_MODELS_DIR, 'deploy.prototxt'), os.path.join(DNN_MODELS_DIR, 'res10_300x300_ssd_iter_140000_fp16.caffemodel'))
db_manager = SQLiteDatabaseManager()

if 'page' not in st.session_state:
    st.session_state.page = 'Home'
if 'live_frame' not in st.session_state:
    st.session_state.live_frame = None
if 'username' not in st.session_state:
    st.session_state.username = ''

st.sidebar.title("Navigation")
page = st.sidebar.radio("Go to", ["Home", "Register", "Authenticate", "History"])

if page == "Home":
    st.title("Face Unlock System")
    st.write("Welcome to the face unlock application. Use the sidebar to navigate.")
    st.info("This application supports webcam authentication and registration, as well as a history log.")

elif page == "Register":
    st.title("User Registration")
    st.info("Enter a username and use your webcam to take a photo. A clear, well-lit photo works best.")
    
    username = st.text_input("Enter a username to register:")
    st.session_state.username = username
    
    registration_mode = st.radio("Choose registration method:", ["Webcam", "Upload Photo"])
    
    if registration_mode == "Webcam":
        captured_photo = st.camera_input("Take a picture to register")
        if captured_photo is not None:
            if not username:
                st.error("Please enter a username.")
            else:
                file_bytes = np.asarray(bytearray(captured_photo.read()), dtype=np.uint8)
                img = cv2.imdecode(file_bytes, 1)

                face, box = detect_and_align_face(img, face_detector)
                if face is not None:
                    embedding = get_face_embedding(face, face_embedding_model)
                    success, message = db_manager.register_user(username, embedding.tobytes())
                    if success:
                        st.success(f"Registration successful for user '{{username}}'!")
                    else:
                        st.error(message)
                else:
                    st.warning("No face detected in the captured image. Please try again.")
    
    elif registration_mode == "Upload Photo":
        uploaded_file = st.file_uploader("Upload a photo to register", type=["jpg", "jpeg", "png"])
        if uploaded_file is not None:
            if not username:
                st.error("Please enter a username.")
            else:
                file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
                img = cv2.imdecode(file_bytes, 1)
                st.image(img, channels="BGR", caption="Uploaded Photo")

                face, box = detect_and_align_face(img, face_detector)
                if face is not None:
                    embedding = get_face_embedding(face, face_embedding_model)
                    success, message = db_manager.register_user(username, embedding.tobytes())
                    if success:
                        st.success(f"Registration successful for user '{{username}}'!")
                    else:
                        st.error(message)
                else:
                    st.warning("No face detected in the uploaded image. Please try again.")


elif page == "Authenticate":
    st.title("User Authentication")
    st.info("Authenticate using your webcam. A liveness check will be performed.")

    auth_username = st.text_input("Enter username for authentication:")
    captured_photo = st.camera_input("Take a picture to authenticate")

    if captured_photo is not None:
        if not auth_username:
            st.error("Please enter a username.")
        else:
            stored_embedding_bytes = db_manager.get_user_embedding(auth_username)
            if stored_embedding_bytes is None:
                st.error(f"User '{{auth_username}}' not found.")
                db_manager.log_event(auth_username, "Authentication", False)
            else:
                file_bytes = np.asarray(bytearray(captured_photo.read()), dtype=np.uint8)
                live_frame = cv2.imdecode(file_bytes, 1)

                face, box = detect_and_align_face(live_frame, face_detector)
                if face is not None:
                    is_live, liveness_message = check_liveness(live_frame)
                    if is_live:
                        live_embedding = get_face_embedding(face, face_embedding_model)
                        is_match, distance = verify_face(stored_embedding_bytes, live_embedding)
                        if is_match:
                            st.success(f"Authentication Successful! Distance: {{distance:.2f}} (Lower is better)")
                            db_manager.log_event(auth_username, "Authentication", True)
                        else:
                            st.error(f"Authentication Failed. Distance: {{distance:.2f}}")
                            db_manager.log_event(auth_username, "Authentication", False)
                    else:
                        st.warning(f"Liveness check failed: {{liveness_message}}")
                        db_manager.log_event(auth_username, "Authentication", False)
                else:
                    st.warning("No face detected in the image.")
                    db_manager.log_event(auth_username, "Authentication", False)

elif page == "History":
    st.title("Activity History")
    
    history = db_manager.get_history()
    
    if history:
        st.write("---")
        for username, event_type, timestamp, success in history:
            status = "✅ Success" if success else "❌ Failed"
            st.write(f"**User:** {{username}} | **Event:** {{event_type}} | **Timestamp:** {{timestamp}} | **Status:** {{status}}")
        st.write("---")
    else:
        st.info("No activity history found.")

    st.write("---")
    st.header("Registered Users")
    users = db_manager.get_all_users()
    if users:
        for username, reg_date in users:
            st.write(f"**User:** {{username}} | **Registration Date:** {{reg_date}}")
    else:
        st.info("No users registered yet.")
"""

# Write the generated code to files
with open(os.path.join(APP_ROOT_KAGGLE, 'utils.py'), "w") as f:
    f.write(utils_code)
with open(os.path.join(APP_ROOT_KAGGLE, 'app.py'), "w") as f:
    f.write(app_code)
print("utils.py and app.py generated successfully.")

# --- Phase 3: Running Streamlit App with ngrok (NOHUP + External Log Monitoring) ---
print("\n--- Phase 3: Running Streamlit App ---")
print("Attempting to set ngrok auth token...")
try:
    # IMPORTANT: Ensure your ngrok auth token is correct and updated if needed.
    ngrok.set_auth_token("30NVTR3IDN5fDUVZ6XegDSSj2ho_SCnczzHNLMG43LqgYpAK") # Replace with your actual ngrok token
    print("ngrok auth token set successfully.")
except Exception as e:
    print(f"Error setting ngrok auth token: {e}")
    print("Please ensure your ngrok token is correct. Exiting.")
    exit()

# Define log file path
log_file_path = "streamlit_logs.txt"

# Start Streamlit using nohup for detachment and redirect all output to a file
print("Starting Streamlit app in a detached process using nohup...")
# Construct the command carefully, quoting paths if they might have spaces (though not in this case)
streamlit_cmd_parts = [
    "nohup",
    sys.executable, "-m", "streamlit", "run", os.path.join(APP_ROOT_KAGGLE, 'app.py'),
    "--server.port", "8501",
    "--server.enableCORS", "false",
    "--browser.gatherUsageStats", "false",
    "--server.headless", "true",
    "&>", # Redirect both stdout and stderr
    log_file_path,
    "&" # Run in background
]
# Join the parts with spaces to form the full shell command string
shell_command = " ".join(streamlit_cmd_parts)
os.system(shell_command) # Execute as a shell command

print(f"Streamlit app launched in background, logging to {log_file_path}")

# Now, we monitor the log file directly by tailing it
print("Monitoring Streamlit app initialization from log file...")
max_wait_time = 120 # seconds, increased tolerance
check_interval = 2 # seconds
start_time = time.time()
app_online = False

while time.time() - start_time < max_wait_time:
    # Guaranteed print every check_interval
    if os.path.exists(log_file_path):
        try:
            # Read from the end of the file
            with open(log_file_path, "r") as f:
                f.seek(0, os.SEEK_END)
                fsize = f.tell()
                f.seek(max(0, fsize - 1500), os.SEEK_SET) # Read last 1.5KB
                tail = f.read()
                
                # Print the tail for a guaranteed heartbeat and recent logs
                print(f"\n--- Current Streamlit Log Tail ({time.strftime('%H:%M:%S', time.localtime())}) ---\n{tail}\n---------------------------------------\n", flush=True)
                
                # Check for the "You can now view..." message in the tail
                if "You can now view your Streamlit app in your browser" in tail:
                    print("\nStreamlit app detected as online in logs.")
                    app_online = True
                    break
        except Exception as e:
            print(f"Error reading log tail: {e}", flush=True)
    else:
        print(f"\n--- Waiting for Streamlit Log File ({time.strftime('%H:%M:%S', time.localtime())}) ---", flush=True) 

    time.sleep(check_interval)

if not app_online:
    print("\n--- Streamlit app did not report readiness within the expected time. ---")
    print("It might still be initializing or have encountered an early error. Ngrok may still fail.")

print("Attempting to create ngrok tunnel...")
public_url = None
try:
    public_url = ngrok.connect(8501)
    print("ngrok tunnel created successfully.")
    print(f"Your ngrok URL is: {public_url}")
    print("\nIf the URL shows 'ERR_NGROK_3200', the Streamlit app is likely still offline or crashed. Please check the Streamlit logs.")
except Exception as e:
    print(f"Error starting ngrok: {e}")

# --- CRITICAL: Keep this cell running to keep the Streamlit app alive on Kaggle! ---
print("\n--- IMPORTANT: Keep this Kaggle cell running to keep your Streamlit app alive! ---")
print(f"Access your Face Unlock app at the ngrok URL: {public_url}")
print("To terminate the app and free resources, manually press the 'Stop' button in Kaggle.")

try:
    # Use a dummy loop to keep the Kaggle kernel alive.
    # The actual Streamlit process is now detached via nohup.
    while True:
        # Periodically check the log file for any late errors or a crash
        if os.path.exists(log_file_path):
            try:
                with open(log_file_path, "r") as f:
                    f.seek(0, os.SEEK_END) # Go to end of file
                    fsize = f.tell()
                    # Read only the very latest tail
                    f.seek(max(0, fsize - 200), os.SEEK_SET) 
                    latest_log_output = f.read()
                    if "Traceback (most recent call last):" in latest_log_output:
                        print("\n--- Streamlit process detected a Python error or crash! ---", flush=True)
                        print(f"Latest log output: {latest_log_output}", flush=True)
                        break # Break the loop to signal crash
            except Exception as e:
                print(f"Error checking log file during runtime: {e}", flush=True)
        
        time.sleep(30) # Sleep to prevent busy-waiting
        print(f"[{time.strftime('%H:%M:%S', time.localtime())}] Kaggle cell active, Streamlit app should be running. Check ngrok URL.", flush=True)
except KeyboardInterrupt:
    print("\nKaggle cell interrupted. Attempting cleanup...", flush=True)
finally:
    # Cleanup processes launched by the shell (nohup)
    print("Performing post-interruption cleanup...", flush=True)
    !fuser -k -9 8501/tcp || true
    !pkill -f ngrok || true
    !killall -q streamlit || true # Ensure any running streamlit processes are killed
    print("Cleanup commands executed.", flush=True)

    # Kill ngrok tunnel (if it was ever established)
    if ngrok.get_tunnels(): # Check if any tunnels are active
        print("Killing ngrok tunnel(s)...", flush=True)
        ngrok.kill()
    else:
        print("No active ngrok tunnels found to kill.", flush=True)
    
    print("Clean shutdown complete. Resources released.", flush=True)

print("\n--- Full Streamlit Logs (after execution/interruption) ---")
# Ensure all logs are flushed and read at the end for final diagnosis
if os.path.exists(log_file_path):
    !cat {log_file_path}
else:
    print("Streamlit log file not found.")

--- Cleanup: Shutting down old processes... ---
Cleanup complete. Starting new session.

--- Setup: Installing/Upgrading dependencies. This will take some time. ---
Dependency installation/upgrade attempt complete.
TensorFlow: Memory growth enabled for GPUs.
Verifying pyngrok installation...
pyngrok imported successfully.

--- Step 1: Preparing files for app launch ---
Cleaning up old database file...
Old database file '/kaggle/working/face_unlock_app/face_db.db' removed successfully.
Attempting to copy liveness model from: /kaggle/input/lma/pytorch/default/1/liveness_model.h5 to /kaggle/working/face_unlock_app/models/liveness_model.h5
Attempting to copy face embedding model from: /kaggle/input/lma/pytorch/default/1/face_embedding_model.h5 to /kaggle/working/face_unlock_app/models/face_embedding_model.h5
Attempting to copy face detector caffemodel from: /kaggle/input/lma/pytorch/default/1/res10_300x300_ssd_iter_140000_fp16.caffemodel to /kaggle/working/face_unlock_app/models/dnn_face_d

2025-07-30 22:27:17.376 
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://34.55.202.18:8501


--- Current Streamlit Log Tail (22:27:18) ---

---------------------------------------


--- Current Streamlit Log Tail (22:27:20) ---

---------------------------------------


--- Current Streamlit Log Tail (22:27:22) ---

---------------------------------------


--- Current Streamlit Log Tail (22:27:24) ---

---------------------------------------


--- Current Streamlit Log Tail (22:27:26) ---

---------------------------------------


--- Current Streamlit Log Tail (22:27:28) ---

---------------------------------------


--- Current Streamlit Log Tail (22:27:30) ---

---------------------------------------


--- Current Streamlit Log Tail (22:27:32) ---

---------------------------------------


--- Current Streamlit Log Tail (22:27:34) ---

---------------------------------------


--- Current Streamlit Log 

2025-07-30 22:29:29.086875: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753914569.112935     238 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753914569.121092     238 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753914569.143414     238 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753914569.143439     238 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753914569.143442     238 computation_placer.cc:177] computation placer alr

2019640/2019640 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
[22:29:47] Kaggle cell active, Streamlit app should be running. Check ngrok URL.


I0000 00:00:1753914600.483929     265 service.cc:152] XLA service 0x7ae2b0003720 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753914600.483963     265 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1753914600.483969     265 service.cc:160]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1753914601.074950     265 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753914605.036144     265 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


[22:30:17] Kaggle cell active, Streamlit app should be running. Check ngrok URL.
[22:30:47] Kaggle cell active, Streamlit app should be running. Check ngrok URL.
[22:31:17] Kaggle cell active, Streamlit app should be running. Check ngrok URL.
[22:31:47] Kaggle cell active, Streamlit app should be running. Check ngrok URL.

Kaggle cell interrupted. Attempting cleanup...
Performing post-interruption cleanup...
Adding missing 'registration_date' column to 'users' table.
  Stopping...
8501/tcp:              218
Cleanup commands executed.
No active ngrok tunnels found to kill.
Clean shutdown complete. Resources released.

--- Full Streamlit Logs (after execution/interruption) ---
